In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pf_fnc as pf

In [2]:
# データ読み込み
r, x, b, bc, P, Q, n = pf.common_parameter()

# 初期化
theta = np.zeros(4)
V = np.zeros(4)
Y = np.zeros((4,4), dtype=np.complex)
cnt = 0

p = np.array([P[1], Q[1], P[2], Q[2], P[3]])

fnc_v = np.zeros(5)

In [3]:
# ノードアドミタンス行列
for jj in range(n):
    for ii in range(n):
        summ=0.0
        temp=0.0 # テンポラリ変数初期化
        
        if (ii==jj):
            for jj2 in range(n):
                if (ii!=jj2): # 自ノード除去
                    temp = 1.0/(r[ii,jj2] + 1.0j*x[ii,jj2]) + 1.0j*b[ii,jj2]/2
                    summ = summ + temp
            
            Y[ii,jj] = summ + bc[ii]*1.0j
            
        if (ii!=jj):
            Y[ii,jj] = -1.0/(r[ii,jj] + 1.0j*x[ii,jj])
            
            
G = Y.real
B = Y.imag

display(Y)

array([[ 0.03998401-1.79920032j, -0.03998401+1.99920032j,
        -0.        +0.j        , -0.        +0.j        ],
       [-0.03998401+1.99920032j,  0.11995202-5.59760096j,
        -0.07996801+3.99840064j, -0.        +0.j        ],
       [-0.        +0.j        , -0.07996801+3.99840064j,
         0.11995202-5.59760096j, -0.03998401+1.99920032j],
       [-0.        +0.j        , -0.        +0.j        ,
        -0.03998401+1.99920032j,  0.03998401-1.79920032j]])

In [4]:
## 潮流計算
dfPi_dthetaj = np.zeros((n,n))
dfQi_dthetaj = np.zeros((n,n))
dfPi_dVj = np.zeros((n,n))
dfQi_dVj = np.zeros((n,n))
Jacobian = np.zeros((5,5))

# フラットスタート
V = np.array([1.0, 1.0, 1.0, 1.0]) # 複素電圧の大きさ[pu]
theta = np.array([0.0, 0.0, 0.0, 0.0]) # 位相角[rad]

# 未知数ベクトル
v = [0.0, 1.0, 0.0, 1.0, 0.0]

# ミスマッチベクトルの無限大ノルムが閾値0.001以下になれば計算終了
while np.linalg.norm(p-fnc_v, np.inf) > 0.001:
    # 初期化
    fP = np.zeros(n) # fPiの配列
    fQ = np.zeros(n) # fQiの配列
    
    # fPiの計算
    for ii in range(n):
        summ=0.0 # テンポラリ変数の初期化
        
        for jj in range(n):
            summ = summ + V[jj]*(G[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy())\
                                 + B[ii,jj] * np.sin(theta[ii].copy()-theta[jj].copy()))
        
        fP[ii] = V[ii].copy() * summ
    
    # fQiの計算
    for ii in range(n):
        summ=0.0
        
        for jj in range(n):
            summ = summ + V[jj].copy()*(G[ii,jj]*np.sin(theta[ii].copy()-theta[jj].copy())\
                                 - B[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy()))
        
        fQ[ii] = V[ii].copy() * summ
        
    # ヤコビアン行列の各成分の計算
    for jj in range(n):
        for ii in range(n):
            if (ii==jj):
                dfPi_dthetaj[ii,jj] = -V[ii].copy()**2 * B[ii,jj] - fQ[ii]
                dfQi_dthetaj[ii,jj] = -V[ii].copy()**2 * G[ii,jj] + fP[ii]
                dfPi_dVj[ii,jj] = V[ii].copy() * G[ii,jj] + fP[ii]/V[ii]
                dfQi_dVj[ii,jj] = -V[ii].copy() * B[ii,jj] + fQ[ii]/V[ii]
            if (ii!=jj):
                dfPi_dthetaj[ii,jj] = V[ii].copy()*V[jj].copy()*(G[ii,jj]*np.sin(theta[ii].copy()-theta[jj].copy())\
                                                   -B[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy()))
                dfQi_dthetaj[ii,jj] = -V[ii].copy()*V[jj].copy()*(G[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy())\
                                                    +B[ii,jj]*np.sin(theta[ii].copy()-theta[jj].copy()))
                dfPi_dVj[ii,jj] = V[ii].copy() * (G[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy())\
                                           -B[ii,jj]*np.sin(theta[ii].copy()-theta[jj].copy()))
                dfQi_dVj[ii,jj] = V[ii].copy() * (-G[ii,jj]*np.sin(theta[ii].copy()-theta[jj].copy())\
                                           -B[ii,jj]*np.cos(theta[ii].copy()-theta[jj].copy()))
    
    Jacobian = np.array([[dfPi_dthetaj[1,1], dfPi_dVj[1,1], dfPi_dthetaj[1,2], dfPi_dVj[1,2], 0],
                         [dfQi_dthetaj[1,1], dfQi_dVj[1,1], dfQi_dthetaj[1,2], dfQi_dVj[1,2], 0],
                         [dfPi_dthetaj[2,1], dfPi_dVj[2,1], dfPi_dthetaj[2,2], dfPi_dVj[2,2], dfPi_dthetaj[2,3]],
                         [dfQi_dthetaj[2,1], dfQi_dVj[2,1], dfQi_dthetaj[2,2], dfQi_dVj[2,2], dfQi_dthetaj[2,3]],
                         [0, 0, dfPi_dthetaj[3,2], dfPi_dVj[3,2], dfPi_dthetaj[3,3]]])
    
    fnc_v = np.array([fP[1], fQ[1], fP[2], fQ[2], fP[3]])
    temp = v + np.dot(np.linalg.inv(Jacobian),(p-fnc_v))
    v = temp.copy()
    
    V = [V[0], v[1], v[3], V[3]]
    theta = [theta[0], v[0], v[2], v[4]]
    cnt = cnt+1

In [5]:
# 結果表示
display(Y)
display(cnt)
display(Jacobian)
display(v)
display(p-fnc_v)
print('計算結果')
print('ノード電圧')
display(V)
print('位相差')
display(theta)

array([[ 0.03998401-1.79920032j, -0.03998401+1.99920032j,
        -0.        +0.j        , -0.        +0.j        ],
       [-0.03998401+1.99920032j,  0.11995202-5.59760096j,
        -0.07996801+3.99840064j, -0.        +0.j        ],
       [-0.        +0.j        , -0.07996801+3.99840064j,
         0.11995202-5.59760096j, -0.03998401+1.99920032j],
       [-0.        +0.j        , -0.        +0.j        ,
        -0.03998401+1.99920032j,  0.03998401-1.79920032j]])

6

array([[ 5.84783165, -0.48326799, -3.96398358, -0.08342309,  0.        ],
       [-0.71888593,  5.27132057,  0.07548272, -3.98050822,  0.        ],
       [-3.96383176, -0.07582053,  5.85079497, -0.48307501, -1.88696321],
       [ 0.08307359, -3.98172365, -0.71894924,  5.27288912,  0.63587565],
       [ 0.        ,  0.        , -1.9108791 , -0.63855088,  1.9108791 ]])

array([-0.30906882,  0.99554673, -0.31000323,  0.99580616, -0.00491896])

array([ 3.88866441e-07, -1.60358193e-06,  1.96402684e-07, -1.61669288e-06,
        9.71912725e-05])

計算結果
ノード電圧


[1.0, 0.9955467346192863, 0.9958061606450417, 1.0]

位相差


[0.0, -0.3090688235614156, -0.31000322980273026, -0.004918956090594681]

In [6]:
# ブランチ潮流の計算
I_dash, Power = pf.node_calc(V, theta, r, x, b, n)

print('I_dash:')
print(I_dash)
print('Power: P[i,j]+jQ[i,j]')
print(Power)

I_dash:
[[ 0.        +0.j          0.60745633-0.29110095j  0.        +0.j
   0.        +0.j        ]
 [-0.66801976-0.09857405j  0.        +0.j         -0.02641965-0.09490549j
   0.        -0.j        ]
 [ 0.        -0.j         -0.0342403 -0.09476587j  0.        +0.j
  -0.66030941-0.0983603j ]
 [ 0.        -0.j          0.        +0.j          0.59856915-0.29130502j
   0.        +0.j        ]]
Power: P[i,j]+jQ[i,j]
[[ 0.        +0.j          0.60745633+0.29110095j  0.        +0.j
   0.        +0.j        ]
 [-0.60368331+0.295773j    0.        -0.j          0.00368327+0.09800631j
   0.        +0.j        ]
 [ 0.        +0.j         -0.0036832 +0.10027174j  0.        -0.j
  -0.59631682+0.2938692j ]
 [ 0.        +0.j          0.        -0.j          0.59999482+0.28835717j
   0.        -0.j        ]]
